In [1]:
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
from scipy import fftpack
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm, tqdm_notebook
import math

from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from keras_tqdm import TQDMNotebookCallback
from tensorboard.plugins.hparams import api as hp
from livelossplot.tf_keras import PlotLossesCallback

import autosklearn.regression
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave

from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
import timeit
from skimage.transform import resize
from timeit import default_timer as timer
from datetime import timedelta
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.keras.models import Model, load_model
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import plot_model

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import bayes_mixed_glm as glm

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV

from spectrum import arburg, arma2psd, pburg
import pylab
from scipy.signal import find_peaks
from scipy.integrate import simps
from IPython.core.display import display, HTML
from scipy.io import loadmat

/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/pyparsing.py:3168: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
eeglab_path = '/home/raquib/Documents/MATLAB/eeglab2019_0/functions/'
octave.addpath(eeglab_path + 'guifunc')
octave.addpath(eeglab_path + 'popfunc')
octave.addpath(eeglab_path + 'adminfunc')
octave.addpath(eeglab_path + 'sigprocfunc')
octave.addpath(eeglab_path + 'miscfunc')

'.:/home/raquib/Documents/MATLAB/eeglab2019_0/functions/sigprocfunc:/home/raquib/Documents/MATLAB/eeglab2019_0/functions/adminfunc:/home/raquib/Documents/MATLAB/eeglab2019_0/functions/popfunc:/home/raquib/Documents/MATLAB/eeglab2019_0/functions/guifunc:/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/oct2py:/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/octave_kernel:/usr/lib/x86_64-linux-gnu/octave/4.2.2/site/oct/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/api-v51/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/x86_64-pc-linux-gnu:/usr/share/octave/4.2.2/site/m:/usr/share/octave/site/api-v51/m:/usr/share/octave/site/m:/usr/share/octave/site/m/startup:/usr/lib/x86_64-linux-gnu/octave/4.2.2/oct/x86_64-pc-linux-gnu:/usr/share/octave/4.2.2/m:/usr/share/octave/4.2.2/m/audio:/usr/share/octave/4.2.2/m/debian:/usr/share/octave/4.2.2/m/deprecated:/usr/share/octave/4.2.2/m/elfun:/usr/share/octave/4.2.2/m/general:/usr/share/octave/4.2

In [3]:
eegs = ['data/original/sub03/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub03/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub03/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub03/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub03/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub03/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub04/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub04/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub04/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
#  'data/original/sub05/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',  # NO MEP
#  'data/original/sub05/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',  # NO MEP
 'data/original/sub05/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub06/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub06/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub06/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub06/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub07/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub07/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub07/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub08/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub09/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub10/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub10/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub10/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub10/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub11/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub12/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub12/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub12/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub12/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub12/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
#  'data/original/sub13/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat', # Left handed
# 'data/original/sub14/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',   # BAD MEP
#  'data/original/sub14/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat', # BAD MEP
# 'data/original/sub14/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',   # BAD MEP
 'data/original/sub15/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub15/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub15/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub16/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub16/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub16/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub17/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub18/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub18/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub18/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
 'data/original/sub19/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
 'data/original/sub19/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
 'data/original/sub19/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat']


rejected_trials = [
    {'sub': 'sub03', 'exp': 'exp02', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub03', 'exp': 'exp02', 'run': 'r2', 'trial': '*'},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': '*'},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub09', 'exp': 'exp01', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r2', 'trial': '*'},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': '*'},
    {'sub': 'sub11', 'exp': 'exp01', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub11', 'exp': 'exp02', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub12', 'exp': 'exp01', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub15', 'exp': 'exp01', 'run': 'r1', 'trial': '*'},
    {'sub': 'sub08', 'exp': 'exp03', 'run': 'r1', 'trial': 0},
    {'sub': 'sub09', 'exp': 'exp01', 'run': 'r1', 'trial': 0},
    {'sub': 'sub11', 'exp': 'exp02', 'run': 'r1', 'trial': 14},
    {'sub': 'sub11', 'exp': 'exp02', 'run': 'r1', 'trial': 38},
    {'sub': 'sub12', 'exp': 'exp01', 'run': 'r1', 'trial': 0},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 14},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 36},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 44},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 11},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 15},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 16},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 17},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 24},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 25},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 26},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 27},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 28},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 29},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 30},
    {'sub': 'sub12', 'exp': 'exp03', 'run': 'r1', 'trial': 32},
    {'sub': 'sub03', 'exp': 'exp01', 'run': 'r2', 'trial': 0},
    {'sub': 'sub03', 'exp': 'exp01', 'run': 'r3', 'trial': 0},
    {'sub': 'sub03', 'exp': 'exp01', 'run': 'r3', 'trial': 5},
    {'sub': 'sub03', 'exp': 'exp01', 'run': 'r3', 'trial': 6},
    {'sub': 'sub03', 'exp': 'exp01', 'run': 'r3', 'trial': 8},
    {'sub': 'sub04', 'exp': 'exp01', 'run': 'r2', 'trial': 16},
    {'sub': 'sub04', 'exp': 'exp01', 'run': 'r2', 'trial': 21},
    {'sub': 'sub04', 'exp': 'exp01', 'run': 'r2', 'trial': 22},
    {'sub': 'sub04', 'exp': 'exp01', 'run': 'r2', 'trial': 23},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': 3},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': 4},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': 5},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': 26},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': 3},
    {'sub': 'sub05', 'exp': 'exp01', 'run': 'r3', 'trial': 3},
    {'sub': 'sub06', 'exp': 'exp01', 'run': 'r2', 'trial': 37},
    {'sub': 'sub06', 'exp': 'exp02', 'run': 'r1', 'trial': 0},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 0},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 3},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 11},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 45},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 46},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 47},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 48},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 49},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 38},
    {'sub': 'sub07', 'exp': 'exp01', 'run': 'r2', 'trial': 24},
    {'sub': 'sub08', 'exp': 'exp02', 'run': 'r1', 'trial': 0},
    {'sub': 'sub09', 'exp': 'exp01', 'run': 'r1', 'trial': 0},
    {'sub': 'sub09', 'exp': 'exp01', 'run': 'r1', 'trial': 1},
    {'sub': 'sub10', 'exp': 'exp01', 'run': 'r1', 'trial': 2},
    {'sub': 'sub10', 'exp': 'exp01', 'run': 'r1', 'trial': 26},
    {'sub': 'sub10', 'exp': 'exp01', 'run': 'r1', 'trial': 16},
    {'sub': 'sub10', 'exp': 'exp01', 'run': 'r1', 'trial': 15},
    {'sub': 'sub10', 'exp': 'exp01', 'run': 'r1', 'trial': 12},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r2', 'trial': 37},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r2', 'trial': 42},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 23},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 24},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 25},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 26},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 27},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 28},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 29},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 30},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 31},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 32},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 33},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 34},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 35},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 36},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 37},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 38},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 39},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 40},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 41},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 42},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 43},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 44},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 45},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 46},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 47},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 48},
    {'sub': 'sub10', 'exp': 'exp02', 'run': 'r3', 'trial': 49},
    {'sub': 'sub12', 'exp': 'exp01', 'run': 'r1', 'trial': 0},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 44},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 36},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 14},
    {'sub': 'sub12', 'exp': 'exp01', 'run': 'r1', 'trial': 0},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 44},
    {'sub': 'sub12', 'exp': 'exp02', 'run': 'r1', 'trial': 14},
    {'sub': 'sub09', 'exp': 'exp01', 'run': 'r1', 'trial': 0},
    {'sub': 'sub16', 'exp': 'exp01', 'run': 'r3', 'trial': 14},
    {'sub': 'sub15', 'exp': 'exp01', 'run': 'r2', 'trial': 36},
    {'sub': 'sub15', 'exp': 'exp01', 'run': 'r2', 'trial': 14},
    {'sub': 'sub19', 'exp': 'exp01', 'run': 'r1', 'trial': 0}
]

# Basic I/O and conversions.

In [4]:
def read_eeg(path):
    eeg = octave.pop_loadset(path)
    new_trial_list = []
    for i in range(eeg.data.shape[2]):
        trial = eeg.data[:, :, i]
        time = np.linspace(-1000, 1000, num=trial.shape[1])
        trial = pd.DataFrame(np.transpose(trial), columns=eeg.chanlocs.labels[0])
        trial['time'] = time
        new_trial_list.append(trial)
    return new_trial_list

def read_trials_from_mat(filename):
    x = loadmat(filename)
    mat_trials = x['dat'][0][0][3][0]
    trials = []
    time = np.linspace(-1000, 1000, len(mat_trials[0][0]))
    for mat_trial in mat_trials:
        trials.append(pd.DataFrame({'C3': mat_trial[0], 'C4': mat_trial[1], 'time': time}))
    return trials

def crop_trials(trial_list, duration_millis=500, sampling_rate=2048):
    new_trial_list = []
    for trial in trial_list:
        samples_to_pick = duration_millis * sampling_rate / 1000
        new_trial_list.append(trial.tail(int(samples_to_pick)))
    return new_trial_list, samples_to_pick

def crop_mep_region(mep_frame, crop_start=0.211, crop_end=0.4):
    multiplier = 7499 / 1.4998
    start = int(crop_start * multiplier)
    end = int(crop_end * multiplier)
    mep_cropped = mep_frame.iloc[start:end, :]
    time = mep_frame.iloc[start:end, :]
    return mep_cropped

def get_apb_column_name(mep_frame):
    if 'L APB' in mep_frame:
        return 'L APB'
    else:
        return 'APB'

def open_mep_as_df(path):
    fileMep = open(path, "r+")
    mep_frames = fileMep.read().split('\n\n')
    df_list = []
    for mep_frame in mep_frames:
        df_list.append(pd.read_csv(StringIO(mep_frame), '\t'))
    return df_list

def get_mep_path(eeg_path):
    segments = eeg_path.split('/')
    segments[4] = 'mep'
    segments = segments[:-1]
    mep_path = '/'.join(segments) + '/*.txt'
    mep_path = glob.glob(mep_path)[0]
    return mep_path

def calculate_mep_size(mep_frame):
    mep_cropped = crop_mep_region(mep_frame)
    apb_name = get_apb_column_name(mep_frame)
    max_row = mep_frame.iloc[mep_cropped.idxmax(axis=0)[apb_name]]
    min_row = mep_frame.iloc[mep_cropped.idxmin(axis=0)[apb_name]]
    mep_size = max_row[apb_name] - min_row[apb_name]
    return mep_size, min_row, max_row

# Read CMAP file.
def read_cmap(path):
    df = pd.read_excel(path, sheet_name='Raw data', header=None)
    columns = []
    ix = 0
    for r in df.index.values:
        if df[0][r] == 'Channel':
            ix = r
            break
    for c in df.columns.values:
        if (df[c][ix] == 2):
            columns.append(c)
    for r in df.index.values:
        if df[0][r] == 'Time (ms)':
            ix = r
            break
    df2 = df[columns].iloc[np.linspace(ix+1, df.shape[0]-2, df.shape[0]-ix-2)]
    df2['time'] = df[0].iloc[np.linspace(ix+1, df.shape[0]-2, df.shape[0]-ix-2)]
    return df2

# Identify CMAP peaks.
def find_cmap_peaks(df):
    df2 = df[(df['time'] > 105) * (df['time'] < 120)]
    time = df2['time']
    mean_clean = np.mean(df2.drop('time', axis=1), axis=1)
    return mean_clean.max(), mean_clean.min()

def get_cmap_path(eeg_path):
    segments = eeg_path.split('/')
    segments[4] = 'cmap'
    del segments[5]
    segments = segments[:-1]
    cmap_path = '/'.join(segments) + '/*.xlsx'
    cmap_paths = glob.glob(cmap_path)
    if (len(cmap_paths) == 0):
        print('No CMAP found for ' + cmap_path)
        return ''
    return cmap_paths[0]

def get_mep_category_absolute_binary(mep_size):
    if mep_size <= .5:
        return 0
    else:
        return 1
    
def calculate_mep_categories_binary(mep_sizes):
    p1 = np.percentile(mep_sizes, 50)
    cat = np.ones(len(mep_sizes)) * (mep_sizes >= p1)
    return cat

def calculate_mep_categories_cmap_binary(mep_sizes, cmap):
    sizes = []
    for mep_size in mep_sizes:
        sizes.append(mep_size / cmap)
    p1 = np.percentile(sizes, 50)
    cat = np.ones(len(mep_sizes)) * (sizes >= p1)
    return cat

def hjorth_transform(trial):
    ch = trial[['FC5','FC1','C3','CP5','CP1','FC3','C5','C1','CP3']].values
    trial['LTM1'] = np.mean(ch, axis=1)
    ch = trial[['FC6','FC2','C4','CP6','CP2','FC4','C6','C2','CP4']].values
    trial['RTM1'] = np.mean(ch, axis=1)
    ch = trial[['Fp1','AF3','AF7','F1','F3','F5','F7','FC1','FC3', 'FC5']].values
    trial['LTDLPFC'] = np.mean(ch, axis=1)
    ch = trial[['Fp2','AF4','AF8','F2','F4','F6','F8','FC2','FC4', 'FC6']].values
    trial['RTDLPFC'] = np.mean(ch, axis=1)
    ch = trial[['Fz','FCz','Cz','F1','FC1','C1','C2','FC2','F2']].values
    trial['CNTRL'] = np.mean(ch, axis=1)
    ch = trial[['PO7', 'PO5', 'PO3', 'O1']].values
    trial['LTOCC'] = np.mean(ch, axis=1)
    ch = trial[['PO4', 'PO6', 'PO8', 'O2']].values
    trial['RTOCC'] = np.mean(ch, axis=1)
    ch = trial[['PO7', 'PO5', 'PO3', 'POz', 'PO4', 'PO6', 'PO8', 'O1', 'Oz', 'O2']].values
    trial['OCC'] = np.mean(ch, axis=1)
    ch = trial[['P1', 'P3', 'CP1', 'CP3']].values
    trial['LTPAR'] = np.mean(ch, axis=1)
    ch = trial[['P2', 'P4', 'CP2', 'CP4']].values
    trial['RTPAR'] = np.mean(ch, axis=1)
    ch = trial[['P1', 'P3', 'CP1', 'CP3', 'P2', 'P4', 'CP2', 'CP4']].values
    trial['PAR'] = np.mean(ch, axis=1)
    return trial

# MEP latency and duration

In [5]:
def calculate_mep_latency_duration(mep_frame, plot=False, title=None):    
    
    # Calculate prestimulus mean amplitude and std.
    std_limit = 5
    peak_lookup_start_time = .215
    apb_name = get_apb_column_name(mep_frame)
    cropped = crop_mep_region(mep_frame, crop_start=0, crop_end=0.198)[apb_name]
    mean_amp = np.mean(cropped.values)
    std = np.std(cropped)
    
    # Find the index of the first point where amp > 5 SD of mean prestimulus
    df_peak = mep_frame[mep_frame['s'] > peak_lookup_start_time]
    peak_time = 0
    for idx, row in df_peak.iterrows():
        if (row[apb_name] > std_limit * std + mean_amp or row[apb_name] < mean_amp - std_limit * std):
            peak_time = row['s']
            break

    latency_absolute = mep_frame[mep_frame['s'] == peak_time]['s'].values[0]
    latency = (latency_absolute - 0.2)
    
    # Calculate the MEP duration.
    std_limit = 6
    sample_count = mep_frame[mep_frame['s'] < 0.03].shape[0]
    df_mean = crop_mep_region(mep_frame, crop_start=0.26, crop_end=0.3)[apb_name]
    mean_amp = np.mean(df_mean.values)
    std = np.std(df_mean.values)
    consecutive_baseline_count = 0
    return_to_baseline_time = 0
    cropped = crop_mep_region(mep_frame, crop_start=latency_absolute, crop_end=0.3)
    for idx, row in cropped.iterrows():
        if (row[apb_name] < mean_amp + std_limit * std) and (row[apb_name] > mean_amp - std_limit * std):
            if consecutive_baseline_count == 0:
                return_to_baseline_time = row['s']
            consecutive_baseline_count += 1
        else:
            consecutive_baseline_count = 0
        if consecutive_baseline_count > sample_count:
            break
    
    if plot == True:
        plt.figure(figsize=(20,5))
        plt.plot(mep_frame.iloc[0:2000]['s'], mep_frame.iloc[0:2000][apb_name])
        plt.axvspan(latency_absolute, return_to_baseline_time, color='r', alpha=0.15)
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude (mV)')
        if title != None:
            plt.title(title)
    return latency, return_to_baseline_time-0.2


# Inspect latencies.
# for eeg_path in eegs:
#     mep_list = open_mep_as_df(get_mep_path(eeg_path))
#     i = 0
#     for mep_frame in mep_list:
#         calculate_mep_latency(mep_frame, plot=True, title=eeg_path + ' - ' + str(i))
#         i = i+1
#         if i == 2:
#             break

# Power and phase calculations

In [6]:
def blackman_harris_filter(channel, time, cutoffs, fs):
    numtaps = 51
    b = signal.firwin(numtaps, cutoffs, window='blackmanharris', fs=fs)
    filtered = signal.lfilter(b, 1, channel)
    delay = 0.5 * (numtaps - 1) / fs
    df = pd.DataFrame({
        'time': time-delay,
        'channel': filtered
    })
    return df

def get_phase(channel, time, band, fs=4096, plot=False):
    df_filtered = blackman_harris_filter(channel, time, [0.00001, band[0]], fs)
    df_filtered = blackman_harris_filter(df_filtered['channel'].values, df_filtered['time'].values, [band[1], fs/2-1], fs)    
    hilb = signal.hilbert(df_filtered[(df_filtered['time'] > -750) * (df_filtered['time'] < -1)]['channel'])
    phase = np.angle(hilb, deg=True)
    df_phase = pd.DataFrame(phase, columns=['phase'])
    df_phase['time'] = df_filtered[(df_filtered['time'] > -750) * (df_filtered['time'] < -1)]['time'].values
    
    if plot == True:
        plt.figure()
        plt.plot(df_phase['time'], df_phase['phase'])
        plt.plot(df_phase.iloc[-1]['time'], df_phase.iloc[-1]['phase'], 'o')
        plt.show()
    # df_phase = df_phase[df_phase['time'] <= -20]
    return df_phase.iloc[-1]['phase'] + 180


def get_power(channel, time, crop_start_millis=-550, fs=500, method='welch'):       
    # Crop.
    channel = channel[(time > crop_start_millis) * (time < -1)].values
    time = time[(time > crop_start_millis) * (time < -1)].values

    # Resample.
    ts = time[1] - time[0]
    secs = len(channel) * ts
    secs = secs/1000
    resampled = signal.resample(channel, int(secs*fs))
    resampled_time = np.linspace(time[0], time[-1], len(resampled))
    
    # Filter if gamma region.
    df_filtered = blackman_harris_filter(resampled, resampled_time, [48, 52], fs)
    resampled = df_filtered['channel']
    resampled_time = df_filtered['time']
    
    # PSD.
    try:
        if method == 'pburg':
            # Burgs method
            p = pburg(resampled, 200, sampling=fs, NFFT=4096)
            power = p.psd
            freq = np.linspace(0, fs, len(power))
            df_power = pd.DataFrame({'freq': freq, 'power': np.log(power)})
        else:
            # Welch method
            freq, power = signal.welch(resampled, fs, nperseg=fs/2, nfft=2000)
            freq_res = freq[1] - freq[0]
            df_power = pd.DataFrame({'freq': freq, 'power': np.log(power)})
    except:
        return 0

    return df_power

In [7]:
# plt.ion()
# eeg_path = eegs[30]
# files = glob.glob(os.path.dirname(eeg_path) + "/06-clean-prestimulus-hjorth.mat")
# trials = read_trials_from_mat(os.path.dirname(eeg_path) + "/06-clean-prestimulus-hjorth.mat")
# trial = trials[20]
# df_power = get_power(trial['C3'], trial['time'])
# plt.plot(df_power['freq'], df_power['power'])
# plt.xlim(6, 100)
# plt.show()

# df_power = get_power(trials[5]['C3'], trials[5]['time'], method='pburg')
# plt.plot(df_power['freq'], df_power['power'])
# plt.xlim(6, 100)
# plt.show()

In [8]:
def process_trials(eeg_path):
    plt.ioff()
    
    segments = eeg_path.split('/')
    sub = segments[2]
    exp = segments[3]
    run = segments[5]

#     files = glob.glob(os.path.dirname(eeg_path) + "/06-clean-prestimulus-hjorth.mat")
#     if (len(files) == 0):
#         return
#     trials = read_trials_from_mat(os.path.dirname(eeg_path) + "/06-clean-prestimulus-hjorth.mat")
    trials = pickle.load(open(os.path.dirname(eeg_path) + "/06-clean-prestimulus.p", "rb"))

    # Get CMAP.
    try:
        cmap = read_cmap(get_cmap_path(eeg_path))
        time = cmap['time']
        cmap_mean = np.mean(cmap.drop('time', axis=1), axis=1)
        cmap = pd.DataFrame({'channel': cmap_mean, 'time': time})
        cmap2 = cmap[(cmap['time'] > 105) * (cmap['time'] < 120)]
        cmap_peak_1 = np.max(cmap2['channel'])
        cmap_peak_2 = np.min(cmap2['channel'])
        cmap_found = True
    except:
        cmap_found = False

    # Get MEPs.
    try:
        mep_list = open_mep_as_df(get_mep_path(eeg_path))
        mep_found = True
    except:
        mep_found = False

    for i in range(len(trials)):

        rejected = False
        for rejected in rejected_trials:
            if (sub == rejected['sub']) and (exp == rejected['exp']) and (rejected['run'] == run.split(' ')[2]) and ((rejected['trial'] == i) or (rejected['trial'] == '*')):
                rejected = True
                break

        # Plot C3.
        trial = trials[i]
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(23,4))
        ax1.plot(trial['time'], trial['C3'])
        ax1.set_xlim([np.min(trial['time']), 0])
        ax1.set_title('C3 Hjorth')
        ax1.set_xlabel('Time (ms)')
        ax1.set_ylabel('C3 Amplitude')

        # Plot power.
        try:
            df_power = get_power(trial['C3'], trial['time'], -150, 500, method='welch')
            df_power_in_band = df_power[(df_power['freq'] >= 8) * (df_power['freq'] <= 12)]
            mean_power = np.mean(df_power_in_band['power'])
            df_power_in_band = df_power[(df_power['freq'] >= 13) * (df_power['freq'] <= 30)]
            mean_power2 = np.mean(df_power_in_band['power'])

            ax2.plot(df_power['freq'], df_power['power'])
            ax2.axhline(mean_power, color='r')
            ax2.axhline(mean_power2, color='b')
            ax2.axvspan(8, 12, color='r', alpha=0.1)
            ax2.axvspan(12, 30, color='b', alpha=0.1)
            ax2.set_xlim(5, 200)
            ax2.set_title('Power spectrum')
            ax2.set_xlabel('Frequency (Hz)')
            ax2.set_ylabel('Power density')
        except:
            print('Error calculating power')

        # Plot MEP.
        try:
            if mep_found == True:
                mep = mep_list[i]
                mep_size, mn, mx = calculate_mep_size(mep)
                apb = get_apb_column_name(mep)
                latency, duration = calculate_mep_latency_duration(mep, plot=False)
                ax3.plot(mep['s'], mep[apb])
                ax3.set_xlim([0.15, 0.5])
                ax3.axhline(mn[apb], color='r')
                ax3.axhline(mx[apb], color='r')
                ax3.set_title('MEP: ' + str(mep_size))
                ax3.plot(mn['s'], mn[apb], 'o', color='r')
                ax3.plot(mx['s'], mx[apb], 'o', color='r')
                ax3.axvspan(latency+.2, latency+duration+.2, color='r', alpha=0.1)
                ax3.set_xlabel('Time (s)')
                ax3.set_ylabel('MEP Amplitude')
        except:
            print('No MEP found: ' + sub + '/' + exp + '/' + run)

        # Plot CMAP.
        if cmap_found == True:
            ax4.plot(cmap['time'], cmap['channel'])
            ax4.set_xlim([75, np.max(cmap['time'])])
            ax4.axhline(cmap_peak_1, color='r')
            ax4.axhline(cmap_peak_2, color='r')
            ax4.set_title('CMAP')
            ax4.set_xlabel('Time (ms)')
            ax4.set_ylabel('CMAP Amplitude')

        title = "{'sub': '" + segments[2] + "', 'exp': '" + segments[3] + "', 'run': '" + segments[5].split(' ')[2] + "', 'trial': " + str(i) + "},"
        plt.suptitle(title)
        plt.tight_layout()
        if rejected == True:
            file_path = 'figs/clean/rejected/' + sub + '/' + exp + '/' + run + '/' + str(i) + '.png'
        else:
            file_path = 'figs/clean/accepted/' + sub + '/' + exp + '/' + run + '/' + str(i) + '.png'
        if not os.path.exists(os.path.dirname(file_path)):
            os.makedirs(os.path.dirname(file_path))
        plt.savefig(file_path, format='png')

In [9]:
num_cores = multiprocessing.cpu_count()-2
r = Parallel(n_jobs=num_cores)(delayed(process_trials)(eeg_path) for eeg_path in tqdm(eegs))
print("Done")

100%|██████████| 48/48 [00:00<00:00, 136.38it/s]


Done
